In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2s
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [4]:
inflation = pd.read_csv('Additional.csv', index_col="Period")
inflation.head()

,Yield Curve,Production Index,Housing Starts,Cost of Living Index,Unemployment,Business Confidence,Producer Price Index,Export Price Index,Import Price Index,Basic Price Index for Domestic Goods,USD,JPY,GBP,CNY,OMX Helsinki,Consumer Price Index
Period,,,,,,,,,,,,,,,,
Jan.00,0.62,96.01601,118.4,1466,10.1,13,91.1,110.2,85.9,79.1,1.0137,106.53,0.61834,8.3926,14364.0,98.0
Feb.00,0.53,96.31606,118.2,1476,10.2,20,92.9,113.3,87.2,79.9,0.9834,107.64,0.61466,8.1408,15864.0,98.8
Mar.00,0.45,95.51592,118.4,1485,10.2,17,93.5,114.4,87.9,80.3,0.9643,102.59,0.61063,7.9834,17092.0,99.3
Apr.00,0.22,97.01617,119.3,1490,10.0,15,93.9,115.1,87.4,80.2,0.9470,99.92,0.59802,7.8402,15799.0,99.6
May.00,0.11,101.21690,120.5,1497,9.8,19,93.8,114.6,89.2,81.0,0.9060,98.09,0.60151,7.4996,16344.0,99.9


In [5]:
inflation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 277 entries, Jan.00 to Jan.23
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Yield Curve                           277 non-null    float64
 1   Production Index                      277 non-null    float64
 2   Housing Starts                        277 non-null    float64
 3   Cost of Living Index                  277 non-null    int64  
 4   Unemployment                          277 non-null    float64
 5   Business Confidence                   277 non-null    int64  
 6   Producer Price Index                  277 non-null    float64
 7   Export Price Index                    277 non-null    float64
 8   Import Price Index                    277 non-null    float64
 9   Basic Price Index for Domestic Goods  277 non-null    float64
 10  USD                                   277 non-null    float64
 11  JPY             

In [6]:
X = inflation.drop('Consumer Price Index', axis=1)
y = inflation['Consumer Price Index']


#creating the x and y value for the dataset and creating a test and training set from them

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
scaler = MinMaxScaler()
scaler.fit (X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

inflation_tree = DecisionTreeRegressor(random_state=42,)

inflation_tree.fit(X_train_scaled, y_train)

y_pred_decision = inflation_tree.predict(X_test_scaled)
mse_tree = mse(y_test, y_pred_decision)
mae_tree = mae(y_test, y_pred_decision)
rsq_tree = r2s(y_test, y_pred_decision)
print ('MSE of Decision Tree: ', mse_tree)
print ('MAE of Decision Tree', mae_tree)
print ('R-Squared of Decision Tree', rsq_tree)


MSE of Decision Tree:  0.3558571428571424
MAE of Decision Tree 0.3414285714285709
R-Squared of Decision Tree 0.9978083516445174


In [8]:
scaler.fit (X)
X_scaled = scaler.transform(X)

In [9]:

dt_regressor = DecisionTreeRegressor(random_state=42, max_depth=10, min_samples_leaf=1, criterion='mse', splitter='random')

# Perform 5-fold cross-validation
cv_scores = cross_val_score(dt_regressor, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Convert the scores from negative MSE to positive MSE
cv_scores = -cv_scores

# Print the cross-validation scores
print('CV Scores:', cv_scores)
print('Mean CV Score:', cv_scores.mean())
print('Standard Deviation:', cv_scores.std())

CV Scores: [19.77267857 17.0275     56.55436364 25.63427273 66.888     ]
Mean CV Score: 37.175362987013
Standard Deviation: 20.49572168345582


In [10]:

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=15, min_samples_leaf=1, max_features='auto')

# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf_regressor, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Convert the scores from negative MSE to positive MSE
cv_scores = -cv_scores

# Print the cross-validation scores
print('CV Scores:', cv_scores)
print('Mean CV Score:', cv_scores.mean())
print('Standard Deviation:', cv_scores.std())

CV Scores: [14.76114068  2.1856422  10.2877184   4.61849715 65.37109409]
Mean CV Score: 19.444818502272703
Standard Deviation: 23.378242369410412


In [11]:
from sklearn.linear_model import Lasso
lasso_regressor = Lasso(alpha=0.01, fit_intercept=True, max_iter=1000, normalize=False, random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(lasso_regressor, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Convert the scores from negative MSE to positive MSE
cv_scores = -cv_scores

# Print the cross-validation scores
print('CV Scores:', cv_scores)
print('Mean CV Score:', cv_scores.mean())
print('Standard Deviation:', cv_scores.std())

CV Scores: [0.05499648 0.00344346 0.00120279 0.0033938  0.01650938]
Mean CV Score: 0.015909180877574508
Standard Deviation: 0.02028045486225236


In [16]:
from sklearn.svm import SVR

svr_regressor = SVR(C=10, epsilon=0.01, kernel='linear')

# Perform 5-fold cross-validation
cv_scores = cross_val_score(svr_regressor, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Convert the scores from negative MSE to positive MSE
cv_scores = -cv_scores

# Print the cross-validation scores
print('CV Scores:', cv_scores)
print('Mean CV Score:', cv_scores.mean())
print('Standard Deviation:', cv_scores.std())

CV Scores: [0.3522588  0.86684972 0.14020571 0.02638506 0.12225635]


AttributeError: 'numpy.ndarray' object has no attribute 'median'

In [13]:
import xgboost as XGB

xgb_regressor = XGB.XGBRegressor(learning_rate=0.1, max_depth=5, min_child_weight=1)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(xgb_regressor, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Convert the scores from negative MSE to positive MSE
cv_scores = -cv_scores

# Print the cross-validation scores
print('CV Scores:', cv_scores)
print('Mean CV Score:', cv_scores.mean())
print('Standard Deviation:', cv_scores.std())


CV Scores: [ 8.10877603  5.86686176  4.95655548  0.47631415 61.57336988]
Mean CV Score: 16.19637545988757
Standard Deviation: 22.823819227434324


In [15]:
from sklearn.neural_network import MLPRegressor

mlp_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), activation='relu', solver='lbfgs', max_iter=5000, learning_rate='constant', alpha=0.01, random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(mlp_regressor, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Convert the scores from negative MSE to positive MSE
cv_scores = -cv_scores

# Print the cross-validation scores
print('CV Scores:', cv_scores)
print('Mean CV Score:', cv_scores.mean())
print('Standard Deviation:', cv_scores.std())

CV Scores: [2.29071537e-02 5.13963357e+00 2.55824307e-03 1.57197974e-03
 1.39025145e-01]
Mean CV Score: 1.0611392184335773
Standard Deviation: 2.03988297593114


In [ ]:
# get the feature importances
importances = model.coefs_[0].sum(axis=1)

# normalize the importances
importances /= importances.sum()

# print the importances
for feature, importance in zip(heading, importances):
    print(feature, importance)


Yield Curve 0.38389137760866404
Production Index 0.2338662368453517
Housing Starts 0.3451105995254318
Cost of Living Index -1.9854179332327795
Unemployment 0.30138974626392745
Business Confidence 0.17407492902953747
Producer Price Index 0.035075680965986664
Export Price Index 0.06233464690834232
Import Price Index 0.12546619761555997
Basic Price Index for Domestic Goods 0.16091866014314155
USD 0.14636623981077293
JPY 0.1968600274628737
GBP 0.24810788210461998
CNY 0.3530466274084203
OMX Helsinki 0.21890908154014926


In [5]:
print('Depth of tree:', inflation_tree.tree_.max_depth)
print('Number of nodes in tree:', inflation_tree.tree_.node_count)

Depth of tree: 11
Number of nodes in tree: 311


In [5]:
param_dt = {'max_depth': [2, 4, 6, 8, 10, 11], 'min_samples_leaf': [1, 2, 5, 10, 15, 20, 25], 'criterion': ['mse', 'friedman_mse'], 'splitter': ['best', 'random'], }
grid_dt = GridSearchCV(estimator=inflation_tree, param_grid=param_dt, scoring='neg_mean_squared_error')
grid_result = grid_dt.fit(X_train_scaled, y_train)
print ('Best Param:', grid_result.best_params_ )

Best Param: {'criterion': 'mse', 'max_depth': 10, 'min_samples_leaf': 1, 'splitter': 'random'}


In [ ]:
(random_state=42, max_depth=10, min_samples_leaf=1, criterion='mse', splitter='random')

In [7]:
inflation_tree = DecisionTreeRegressor(random_state=42, max_depth=11, min_samples_leaf=1, criterion='mse', splitter='best')

inflation_tree.fit(X_train_scaled, y_train)

y_pred_decision = inflation_tree.predict(X_test_scaled)
mse_tree = mse(y_test, y_pred_decision)
mae_tree = mae(y_test, y_pred_decision)
rsq_tree = r2s(y_test, y_pred_decision)
print ('MSE of Decision Tree: ', mse_tree)
print ('MAE of Decision Tree', mae_tree)
print ('R-Squared of Decision Tree', rsq_tree)


MSE of Decision Tree:  0.17828571428571438
MAE of Decision Tree 0.254285714285713
R-Squared of Decision Tree 0.9989019762554627


In [8]:

inflation_random = RandomForestRegressor(random_state=42)
inflation_random.fit(X_train_scaled, y_train)
y_pred_rand = inflation_random.predict(X_test_scaled)
mse_rand = mse(y_test, y_pred_rand)
mae_rand = mae(y_test, y_pred_rand)
r2 = r2s(y_test, y_pred_rand)
print ('MSE of Random Forest:',mse_rand )
print ('MAE of Random Forest:',mae_rand )
print ('R-squared', r2)

MSE of Random Forest: 0.29725137142858754
MAE of Random Forest: 0.25228571428572116
R-squared 0.9981692921093959


In [9]:
grid = dict(max_depth = [1, 5, 10, 15], min_samples_leaf = [1, 5, 10, 15, 20], n_estimators = [100, 200, 300], max_features= ['auto', 'sqrt', 'log2'])

forest_est = RandomForestRegressor(random_state = 42)

grid_search = GridSearchCV(estimator=inflation_random, param_grid=grid, scoring='neg_mean_squared_error')

grid_result = grid_search.fit(X_train_scaled, y_train)

# Print out the best result
print("Best result is obtained using", grid_result.best_params_)

Best result is obtained using {'max_depth': 15, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 100}


In [10]:
inflation_random = RandomForestRegressor(n_estimators=200, random_state=42, max_depth=10, min_samples_leaf=1, max_features='auto')
inflation_random.fit(X_train, y_train)
y_pred_rand = inflation_random.predict(X_test)
mse_rand = mse(y_test, y_pred_rand)
mae_rand = mae(y_test, y_pred_rand)
r2 = r2s(y_test, y_pred_rand)
print ('MSE of Random Forest:',mse_rand )
print ('MAE of Random Forest:',mae_rand )
print ('R-squared', r2)

MSE of Random Forest: 0.2553169205601701
MAE of Random Forest: 0.24244551020409924
R-squared 0.9984275574614581


In [11]:
inflation_random = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=15, min_samples_leaf=1, max_features='auto')
inflation_random.fit(X_train, y_train)
y_pred_rand = inflation_random.predict(X_test)
mse_rand = mse(y_test, y_pred_rand)
mae_rand = mae(y_test, y_pred_rand)
r2 = r2s(y_test, y_pred_rand)
print ('MSE of Random Forest:',mse_rand )
print ('MAE of Random Forest:',mae_rand )
print ('R-squared', r2)

MSE of Random Forest: 0.30137754285716073
MAE of Random Forest: 0.2608285714285783
R-squared 0.9981438798983236


In [12]:
from sklearn.linear_model import Lasso

inflation_lasso = Lasso(random_state=42, )
inflation_lasso.fit(X_train_scaled, y_train)
y_pred_lasso = inflation_lasso.predict(X_test_scaled)
mse_lasso = mse(y_test, y_pred_lasso)
mae_lasso = mae(y_test, y_pred_lasso)
r2_lasso = r2s(y_test, y_pred_lasso)
print ('MSE of LASSO:',mse_lasso )
print ('MAE of LASSO Forest:',mae_lasso )
print ('R-Squared of Lasso', r2_lasso)


MSE of LASSO: 22.591925512483385
MAE of LASSO Forest: 4.242300042507948
R-Squared of Lasso 0.8608611422013919


In [13]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search over
param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'max_iter': [1000, 5000, 10000],
}


# Create a GridSearchCV object
grid_search = GridSearchCV(inflation_lasso, param_grid, cv=5)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best parameter combination found
print("Best parameter combination:", grid_search.best_params_)


Best parameter combination: {'alpha': 0.01, 'fit_intercept': False, 'max_iter': 5000, 'normalize': True}


In [14]:
#After manual tuning
from sklearn.linear_model import Lasso

inflation_lasso = Lasso(alpha=0.01, fit_intercept=True, max_iter=1000, normalize=False, random_state=42)
inflation_lasso.fit(X_train_scaled, y_train)
y_pred_lasso = inflation_lasso.predict(X_test_scaled)
mse_lasso = mse(y_test, y_pred_lasso)
mae_lasso = mae(y_test, y_pred_lasso)
r2_lasso = r2s(y_test, y_pred_lasso)
print ('MSE of LASSO:',mse_lasso )
print ('MAE of LASSO Forest:',mae_lasso )
print ('R-Squared of Lasso', r2_lasso)


MSE of LASSO: 0.011009331479272933
MAE of LASSO Forest: 0.06671267319508647
R-Squared of Lasso 0.9999321958720913


In [15]:
inflation_lasso = Lasso(alpha=0.01, fit_intercept=False, max_iter=5000, normalize=True, random_state=42)
inflation_lasso.fit(X_train_scaled, y_train)
y_pred_lasso = inflation_lasso.predict(X_test_scaled)
mse_lasso = mse(y_test, y_pred_lasso)
mae_lasso = mae(y_test, y_pred_lasso)
r2_lasso = r2s(y_test, y_pred_lasso)
print ('MSE of LASSO:',mse_lasso )
print ('MAE of LASSO Forest:',mae_lasso )
print ('R-Squared of Lasso', r2_lasso)

MSE of LASSO: 146.16507737840195
MAE of LASSO Forest: 9.483979663512144
R-Squared of Lasso 0.0998004173996373


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset
data = pd.read_csv('Initial Data.csv')

# Extract the X and Y columns
X = data[['Yield Curve', 'Production Index', 'Housing Starts', 'Cost of Living Index', 'Unemployment']]
y = data['Consumer Price Index']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scale the X data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an SVR model with chosen hyperparameters
svr = SVR(kernel='linear', C=1, epsilon=0.01, )

# Train the SVR model on the training data
svr.fit(X_train_scaled, y_train)

# Evaluate the performance of the model on the testing data
y_pred = svr.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print('MAE:', mae)
print('R-squared:', r2)


MSE: 6.821893575349972
MAE: 2.242964061611064
R-squared: 0.957985410337274


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset
data = pd.read_csv('Initial Data.csv')

# Extract the X and Y columns
X = data[['Yield Curve', 'Production Index', 'Housing Starts', 'Cost of Living Index', 'Unemployment']]
y = data['Consumer Price Index']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scale the X data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an SVR model with chosen hyperparameters
svr = SVR()

# Train the SVR model on the training data
svr.fit(X_train_scaled, y_train)

# Evaluate the performance of the model on the testing data
y_pred = svr.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print('MAE:', mae)
print('R-squared:', r2)


MSE: 10.676229437066187
MAE: 1.5744103005857342
R-squared: 0.9342473766280589


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset
data = inflation

# Extract the X and Y columns
X = data[['Yield Curve', 'Production Index', 'Housing Starts', 'Cost of Living Index', 'Unemployment']]
y = data['Consumer Price Index']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scale the X data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define a dictionary of hyperparameters and their possible values
param_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [0.1, 1, 2, 3, 4, 5, 10], 'epsilon': [0.01, 0.1, 1]}

# Create an SVR model
svr = SVR()

# Use GridSearchCV to find the best combination of hyperparameters
grid_search = GridSearchCV(svr, param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print (best_params)
print (best_model)


{'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}
SVR(C=10, epsilon=0.01, kernel='linear')


In [19]:
svr = SVR (C=10, epsilon=0.01, kernel='linear')

# Train the SVR model on the training data
svr.fit(X_train_scaled, y_train)

# Evaluate the performance of the model on the testing data
y_pred = svr.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print('MAE:', mae)
print('R-squared:', r2)



MSE: 0.0070528073078013195
MAE: 0.04232232962774946
R-squared: 0.9999565632618371


In [20]:
svr = SVR (C=20, epsilon=0.0341, kernel='linear')

# Train the SVR model on the training data
svr.fit(X_train_scaled, y_train)

# Evaluate the performance of the model on the testing data
y_pred = svr.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print('MAE:', mae)
print('R-squared:', r2)



MSE: 0.006657303867834006
MAE: 0.04092340885671218
R-squared: 0.999958999083293
